In [2]:
import re
from nltk import Tree
from pycorenlp import StanfordCoreNLP
import json
import nltk
from nltk.sem.logic import *
import requests

read_expr = nltk.sem.Expression.fromstring
nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

In [69]:
def parser(expression):
    old = expression.replace(' ', '_').replace('>_(', '> (').replace(')_(', ') (').replace(')_)', ') )').replace(')_)', ') )').replace(' (', '(')
    new = ''
    flag = False
    for x in range(0, len(old) - 1):        
        if old[x] == '<':
            flag = True
        if old[x] == '>':
            flag = False
            
        if flag == True:
            if old[x] == '(':
                new += '{'
            elif old[x] == ')':
                new += '}'
            else:
                new += old[x]
        else:
            new += old[x]
    new += old[len(old)-1]
    return new

def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append(pos['pos'])
    return res


def insert_pos_tag(exp, pos):
    count = 0
    res = ''
    for x in range(0, len(exp)):
        if exp[x] == 'S' and exp[x+1]==' ' and exp[x+2] == 'P':
            res += 'S '
            res += pos[count]
            count += 1
            x += 4
        else:
            res += exp[x]
    return res


def direction(exp):
    cont = False
    for x in exp:
        if x == '{':
            cont = True
        elif x == '}':
            cont = False
            continue
        if cont == True:
            continue
        if x == '/':
            return '/'
        elif x == '\\':
            return '\\'
    return False


def is_type_raising(tree):
    tree_string = str(tree)
    
    # check type raising
    exp = tree_string.split('_')[1]
    pattern_1 = r'(.*?)\\(.*?){(.*?)/(.*?)}'
    pattern_2 = r'(.*?)/(.*?){(.*?)\\(.*?)}'
    
    match = False
    if re.search(pattern_1, exp):
        match = True
    elif re.search(pattern_2, exp):
        match = True
        
    sub = []
    for subtree in tree:
        sub.append(subtree)
    if len(sub) == 1 and match:
        return True
    else:
        return False
    

def score(tree):
    tree_string = str(tree)
#     print(tree_string)
    if tree_string[2] == 'L':
        pos = tree_string.split('_')[3]
        word = tree_string.split('_')[5]
        
        #                        #
        # masukin rulenya disini #
        #                        #
        if 'JJ' in pos:
            return '10_'
    else:
        return False

identity = lambda x: x   

def lambda_calculus(tree):
    tree_string = str(tree)
    
    if tree_string[2] == 'L':
        pos = tree_string.split('_')[3]
        word = tree_string.split('_')[5]
        #                        #
        # masukin rulenya disini #
        #                        #
        r_word = ['PRP', 'FW', 'NN', 'LS']
        if pos == 'CC':
            return read_expr(r'CC')
        elif pos in r_word:
            return read_expr(word + '_' + pos + '_' + 'N' + '_' + '0')
        elif 'JJ' in pos:
            return read_expr(word + '_' + 'JJ' + '_' + 'P' + '_' + '2')
        elif 'VB' in pos:
            if '{S[dcl]\\NP}/{S[adj]\\NP}' in tree_string.split('_')[1]:
                return read_expr(r'\x.x')
            else:
                return read_expr(r'\X.' + word + '(X)')
        else:
            return read_expr(r'\x.x')
    
    # ini cuma masukin ke array 
    chunk = []
    sub = []
    
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            sub.append(subtree)
            
            # chunk temp array
            subtree_str_array = str(subtree).split('_')
            if subtree_str_array[0][2] == 'L':
                if subtree_str_array[3] == 'NN':
                    chunk.append(subtree_str_array[5])
    
    
    # chunk noun phrase
    if len(chunk) == 2:
        chunk_str = '+'.join(chunk)
        return read_expr(r'(' + chunk_str + ')')
    
    # error anak 1
    if len(sub) == 1:            
        return lambda_calculus(sub[0])
                        
    # urutan operasi lambda calculusnya    
    first = sub[0]
    second = sub[1]
    
    if is_type_raising(first):
        if direction(str(first).split('_')[1]) == '/':
            x = read_expr(r'\F x.F(x, ' + str(lambda_calculus(first)) + ')')
            y = lambda_calculus(second)
            return ApplicationExpression(x, y).simplify()
    
    if is_type_raising(second):
        if direction(str(second).split('_')[1]) == '/':
            x = lambda_calculus(first)
            y = read_expr(r'\F x.F(x, ' + str(lambda_calculus(second)) + ')')
            return ApplicationExpression(x, y).simplify()
    
        
    length_1 = len(str(sub[0]).split('_')[1].replace('\\', '/').split('/'))
    length_2 = len(str(sub[1]).split('_')[1].replace('\\', '/').split('/'))
    if length_2 > length_1:
        first = sub[1]
        second = sub[0]
    # rekursi
    return deduction(lambda_calculus(first), lambda_calculus(second))

In [70]:
def glue_process(sent):
    url = "http://localhost:5000/ccgParsing"
    data = {"sent": sent}
    r = requests.post(url, data=data)

    res = r.json()
    
    from_res = res['tree']
    pos_tagged = insert_pos_tag(from_res, pos_tag(data['sent']))

    hasil = parser(pos_tagged)

    tree = Tree.fromstring(hasil)
    return lambda_calculus(tree)

In [75]:
def deduction(a, b):
    str_a = str(a)
    str_b = str(b)
    
    #change identity function for adjective
    if(re.search(r'JJ.*', str_a) and re.search(r'NN.*', str_b)):
        adjective_score = str_a.split('_')[3]
        def mapFunction(data):
            idx, x = data
            return adjective_score if idx == 3 else x
        
        str_b = '_'.join( list(map(mapFunction, enumerate(str_b.split('_')))) )
        a = read_expr(r'\x.x')
        b = read_expr(str_b)
    elif(re.search(r'JJ.*', str_a)):
        a = read_expr(r'\x.x')
        a = read_expr(str_a)
    #change identity function for adverb

    if(str_a == 'CC'):
        print('m')
        pass
    if(re.search(r'CC\(', str_a)):
        a = read_expr(r'\x.x')
        b = read_expr(replacer(str_a, str_b))
        
    str_a = str(a)
    str_b = str(b)
    print('a ' + str_a)
    print('b ' + str_b)
    print('hasil ' + str(ApplicationExpression(a, b).simplify()))
    return ApplicationExpression(a, b).simplify()


In [180]:
def replacer(a, b):
    pattern = '(.*?_JJ_.*?_)'
    end_pos = len(re.findall(pattern, a)[0]) + 1
    start_pos = 0
    for x in range(0, end_pos):
        if a[x] == '(' or a[x] == ',':
            start_pos = x + 1
    temp = ''
    for x in range(start_pos, end_pos):
        temp += a[x]
    print(a,b, temp)
    res = str(a).replace(str(temp), str(b))
    print(res)
    return res

In [181]:
# glue_process('the breakfast that the restaurant served daily was excellent')

In [182]:
glue_process('i bought my canon g3 and i have to say i am very satisfied')

a \x.x
b satisfied_JJ_P_2
hasil satisfied_JJ_P_2
a \x.x
b satisfied_JJ_P_2
hasil satisfied_JJ_P_2
a satisfied_JJ_P_2
b i_FW_N_0
hasil satisfied_JJ_P_2(i_FW_N_0)
a \X.say(X)
b satisfied_JJ_P_2(i_FW_N_0)
hasil say(satisfied_JJ_P_2(i_FW_N_0))
a \x.x
b say(satisfied_JJ_P_2(i_FW_N_0))
hasil say(satisfied_JJ_P_2(i_FW_N_0))
a \X.have(X)
b say(satisfied_JJ_P_2(i_FW_N_0))
hasil have(say(satisfied_JJ_P_2(i_FW_N_0)))
a have(say(satisfied_JJ_P_2(i_FW_N_0)))
b i_FW_N_0
hasil have(say(satisfied_JJ_P_2(i_FW_N_0)),i_FW_N_0)
m
a CC
b have(say(satisfied_JJ_P_2(i_FW_N_0)),i_FW_N_0)
hasil CC(have(say(satisfied_JJ_P_2(i_FW_N_0)),i_FW_N_0))
a \x.x
b canon+g3
hasil canon+g3
a \X.bought(X)
b canon+g3
hasil bought(canon+g3)
a bought(canon+g3)
b i_LS_N_0
hasil bought(canon+g3,i_LS_N_0)
CC(have(say(satisfied_JJ_P_2(i_FW_N_0)),i_FW_N_0)) bought(canon+g3,i_LS_N_0) satisfied_JJ_P_2
CC(have(say(bought(canon+g3,i_LS_N_0)(i_FW_N_0)),i_FW_N_0))
a \x.x
b CC(have(say(bought(canon+g3,i_LS_N_0,i_FW_N_0)),i_FW_N_0))
hasil C

<ApplicationExpression CC(have(say(bought(canon+g3,i_LS_N_0,i_FW_N_0)),i_FW_N_0))>

In [183]:
verb = lambda a: lambda b: a + '(' + b + ')'

In [184]:
verb('have')('seat')

'have(seat)'

In [15]:
# rr = i_a('restaurant')

In [10]:
class Lambek:
    def __init__(self, word, score, pos):
        ## private varibale or property in Python
        self.__word = word
        self.__score = score
        self.__pos = pos
        
    def get_word(self):
        return self.__word
    
    def get_score(self):
        return self.__score
    
    def set_score(self, score):
        self.__score = score
    
    def get_pos(self):
        return self.__pos

In [11]:
breakfast = Lambek('breakfast', 0, 'NN')
restaurant = Lambek('restaurant', 0, 'NN')

In [16]:
# ii = i_a(breakfast)

In [17]:
# rr = i_a(restaurant)

In [14]:
served = Lambek('served', 0, 'VB')
daily = Lambek('daily', 1, 'RB')

In [152]:
a = change_identity(daily)(served)

In [153]:
served.get_score()

1

In [155]:
verb('served')('daily')

'served(daily)'

In [158]:
def identity_function(l):
    return l

In [159]:
c_i = lambda x: x
c_i('word')

'word'

In [213]:
from inspect import isfunction

In [214]:
isfunction(identity)

True

In [19]:
# ApplicationExpression.variables()

In [20]:
v = read_expr(r'\x.P(x)')
#v = read_expr(r'\x.x')
print(str(v).split('.'))

['\\x', 'P(x)']


In [21]:
v = map(lambda x: adjective_score if i == 3 else x, [1,2,3,4]);

In [23]:
for i,w in enumerate(['1', '2', '3']):
    print(i, w)

0 1
1 2
2 3


In [176]:
a = 'CC(have(say(satisfied_JJ_P_2(i_FW_N_0)),i_FW_N_0))'
b = 'bought(canon+g3,i_LS_N_0)'
c = 'satisfied_JJ_P_2'

a.replace(c,b)

'CC(have(say(bought(canon+g3,i_LS_N_0)(i_FW_N_0)),i_FW_N_0))'